In [30]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
from keras import optimizers
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

In [65]:
import sys
#sys.path.insert(0, '../../')
sys.path.insert(0, '/home/luka/Developement/accetuation/')
from prepare_data import *

%run ../../prepare_data.py

In [66]:
data = Data('s')
data.generate_data('syllables_word_accetuation_train',
                   'syllables_word_accetuation_test',
                   'syllables_word_accetuation_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/')

LOADING DATA...
LOAD SUCCESSFUL!


In [4]:
content = data._read_content('../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)

In [6]:
pickle_input = open('letters_word_accetuation_predictions.pkl', 'rb')
letters_word_accetuation_predictions = pickle.load(pickle_input)
pickle_input = open('syllables_word_accetuation_predictions.pkl', 'rb')
syllables_word_accetuation_predictions = pickle.load(pickle_input)
pickle_input = open('syllabled_letters_word_accetuation_predictions.pkl', 'rb')
syllabled_letters_word_accetuation_predictions = pickle.load(pickle_input)


pickle_input = open('error_analysis_results/onedirectional_input/letters_word_accetuation_correct_order_reversed_predictions.pkl', 'rb')
letters_word_accetuation_correct_order_predictions = pickle.load(pickle_input)
pickle_input = open('error_analysis_results/onedirectional_input/syllabled_letters_word_accetuation_correct_order_reversed_predictions.pkl', 'rb')
syllables_word_accetuation_correct_order_predictions = pickle.load(pickle_input)
pickle_input = open('error_analysis_results/onedirectional_input/syllables_word_accetuation_correct_order_reversed_predictions.pkl', 'rb')
syllabled_letters_word_accetuation_correct_order_predictions = pickle.load(pickle_input)

In [67]:
ensemble_predictions = np.mean( np.array([ letters_word_accetuation_predictions, syllables_word_accetuation_predictions, syllabled_letters_word_accetuation_predictions, 
                                         letters_word_accetuation_correct_order_predictions, syllables_word_accetuation_correct_order_predictions, syllabled_letters_word_accetuation_correct_order_predictions]), axis=0 )
#ensemble_predictions = np.mean( np.array([ letters_word_accetuation_predictions, syllables_word_accetuation_predictions, syllabled_letters_word_accetuation_predictions]), axis=0 )
#accuracy, errors = data.test_accuracy(ensemble_predictions, data.x_validate, data.x_other_features_validate, data.y_validate, dictionary, feature_dictionary, vowels, syllable_dictionary=syllable_dictionary)
#ensemble_predictions=syllabled_letters_word_accetuation_correct_order_predictions
accuracy, errors = data.test_accuracy(ensemble_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, syllable_dictionary=syllable_dictionary, threshold=0.5, patterns=patterns)
print(accuracy)

589
243
92.71773790771832


In [64]:
589 / len(data.x_test)

0.011314303277113988

In [14]:
int_max = 0

def get_word_length(x_el):
    i = 0
    for el in x_el:
        if el == 0:
            return i
        i += 1
    return 10

#probabilities = np.zeros((10, 10))
#count_results = np.zeros(10)
#for i in range(len(data.y_train)):
#    word_len = get_word_length(data.x_test[i])
    #print(word_len)
    #if word_len > 6:
    #    print(word_len)
    #    print(data.y_test[i])
    
    #
#    for j in range(word_len):
        #print(word_len)
#        if data.y_test[i][word_len - 1 - j] == 1.0:
#            for k in range(word_len):
#                probabilities[j][k] += ensemble_predictions[i][word_len - 1 - k]
            #probabilities[j] += ensemble_predictions[i]
#            count_results[j] += 1
    
#print(probabilities)
#for i in range(len(probabilities)):
#    for j in range(len(probabilities[i])):
#        probabilities[i][j] = probabilities[i][j] / count_results[i]
    
    #probabilities[i] = probabilities[i] / count_results[i]

In [24]:
occurances_of_patterns = {}
for el in zip(data.x_train, data.y_train):
    l = get_word_length(el[0])
    if l in occurances_of_patterns:
        in_occurances = False
        for item in occurances_of_patterns[l]:
            if (item['key'] == el[1]).all():
                item['count'] += 1
                in_occurances = True
                break
        if not in_occurances:
            occurances_of_patterns[l].append({'key': el[1], 'count': 1})
    else:
        occurances_of_patterns[l] = [{'key': el[1], 'count': 1}]


occurances_of_patterns

{1: [{'count': 2575,
   'key': array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])},
  {'count': 82,
   'key': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])}],
 2: [{'count': 41074,
   'key': array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])},
  {'count': 6645,
   'key': array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])},
  {'count': 23,
   'key': array([ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])},
  {'count': 12,
   'key': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])}],
 3: [{'count': 40024,
   'key': array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])},
  {'count': 89275,
   'key': array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])},
  {'count': 4342,
   'key': array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])},
  {'count': 2701,
   'key': array([ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])},
  {'count': 113,
   'key': array([ 1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])},
  {'count': 14,
   'key':

In [61]:
patterns = {}
for key in occurances_of_patterns:
    patterns[key] = []
for key in patterns:
    for item in occurances_of_patterns[key]:
        if item['count'] > len(data.x_train) * 0.001:
            patterns[key].append(item['key'])

In [62]:
patterns

{1: [array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])],
 2: [array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
  array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])],
 3: [array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
  array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
  array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
  array([ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])],
 4: [array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
  array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
  array([ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
  array([ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
  array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
  array([ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.])],
 5: [array([ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
  array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
  array([ 0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]),
  arr

In [28]:
len(data.x_train)  * 0.0001

43.015100000000004